In [48]:
import json
import re
import time
import pandas as pd

In [49]:
df = pd.read_csv('trump_final.csv', index_col=0)

In [50]:
new_date_list = []
for date in df['Dates']:
    date_list = date.split('-')
    date_list[2] = '20'+ date_list[2]
    new_date = '-'.join(date_list)
    struct_time = time.strptime(new_date, "%d-%b-%Y")  # 21-Dec-20
    reformat_date = time.strftime("%Y-%m-%d", struct_time)
    new_date_list.append(reformat_date)
    # print(reformat_date)

In [51]:
new_date_series = pd.Series(data=new_date_list, dtype=str)
# new_date_series.value_counts()
# new_date_series.describe()
# new_date_series.sort_values()

In [52]:
# 将更新后的date插入原df
df.insert(0,'new_date',new_date_series)

In [53]:
# 将Dates列和Link_Content列删除，生成新数组
new_df = df.drop(['Dates', 'Link_Content'], axis=1)

In [54]:
new_columns = ['date', 'lastname', 'text', 'links', 'count', 'rank',
       'type', 'travel_places', 'travel_start_date', 'travel_end_date', 'topics', 'meet_countries', 'link_content']
# 改变df的列标签为全小写
new_df.columns = new_columns

In [55]:
# 按照日期排序，并且重置行索引
new_df = new_df.sort_values('date').reset_index(drop=True)

下面获取travel_places列中的所有地点

In [56]:
place_set = set()  # 集合有去重的功能
for index, value in new_df['travel_places'].items():
    if value not in [' ']:
        place_list = value.split(' ; ')
        place_list.pop(0) # 列表的第一项是空的
        place_set.update(place_list)  # 用列表更新集合中的
place_set_series = pd.Series(list(place_set))
place_set_series.rename('place', inplace=True)
place_set_series.to_csv('place_set.csv')

In [57]:
# 上述文件需要调geopy的api
df_geo = pd.read_csv('place_geo_dict.csv', index_col=0)

In [58]:
geo_dicts = dict()
for i in range(0, len(df_geo)):
    geo_dict_key = df_geo['place'][i]
    geo_dict_value = json.loads(df_geo['place_geo_dict'][i])
    geo_dicts[geo_dict_key] = geo_dict_value

In [59]:
travel_places_series = new_df['travel_places'].copy()
for index, value in travel_places_series.items():
    if value not in [' ']:
        place_list = value.split(' ; ')
        place_list.pop(0) # 列表的第一项是空的
        place_dict = dict()
        for place in place_list:
            place_dict.update(geo_dicts[place])
        json_place = json.dumps(place_dict)
        # new_place = ';'.join(place_list)
        # print(f'place={value}----new_place={new_place}')
        travel_places_series.loc[index] = json_place
new_df['travel_places'] = travel_places_series

In [60]:
new_df.to_csv('trump_with_geo.csv', index=False)